# Librerias

In [26]:
# Librerias para el procesamiento de lenguaje natural

from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk

# Librerias para el procesamiento de archivos xml
import xml.etree.ElementTree as ET

# Librerias de uso general
from collections import defaultdict
import os
import re

# Librerias para tipado de funciones
from typing import List

# Configuración de palabras de parada
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))

# Configuración de stemmer
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package stopwords to /home/nico/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Proceso de preprocesamiento

In [27]:
def preprocess(text: str) -> List[str]:
    # Normalización del texto
    text = text.lower()
    # Tokenización simple a nivel de palabra
    tokens = re.findall(r"\w+", text)
    # Quitar stopwords y aplicar stemming
    processed = [stemmer.stem(token) for token in tokens if token not in stop_words]
    # Retornar texto preprocesado
    return processed

In [28]:
print(preprocess("The cats are running quickly in the gardens"))

['cat', 'run', 'quick', 'garden']


# Ingesta de documentos

In [29]:
# Ruta en la que se encuentran los documentos a procesar
path_docs = "../data/docs-raw-texts"

def load_documents(path_docs: str):
	# Diccionario para almacenar los documentos por id
	docs = {}
	# Se revisan únicamente los archivos relevantes
	for file_name in os.listdir(path_docs):
		if not file_name.endswith(".naf"):
			continue
		# Construccion del arbol de atributos del documento
		tree = ET.parse(os.path.join(path_docs, file_name))
		root = tree.getroot()
		# Id recuperado desde el atributo pupblicId
		doc_id = root.find("nafHeader/public").attrib["publicId"]
		# Titulo desde el atributo title
		title = root.find("nafHeader/fileDesc").attrib.get("title", "")
		# Contenido desde <raw>
		raw_element = root.find("raw")
		content = raw_element.text if raw_element is not None else ""
		# concatenar titulo + contenido para preprocesar
		tokens = preprocess(title +  " " + content)
		docs[doc_id] = tokens
	# Retornar los documentos cargados
	return docs

In [30]:
documents = load_documents(path_docs)

# Construcción de índice invertido

In [33]:
def build_inverted_index(docs):
    # construcción de la estructura de indice invertido
    inverted = defaultdict(set)
    # Iteración sobre los doumentos cargados
    for doc_id, tokens in docs.items():
        # Iteración sobre los tokens de los documentos
        for token in tokens:
            # Población del índice
            inverted[token].add(doc_id)
    # Convertir sets a listas ordenadas
    for token in inverted:
        inverted[token] = sorted(inverted[token])
    # Se retorna el índice construido
    return inverted

In [34]:
# problema, el set no esta quedando ordenado
index = print(build_inverted_index(documents))
print(index)

defaultdict(<class 'set'>, {'watch': ['d010', 'd052', 'd068', 'd069', 'd076', 'd115', 'd139', 'd146', 'd162', 'd165', 'd168', 'd183', 'd193', 'd195', 'd197', 'd200', 'd209', 'd213', 'd214', 'd217', 'd219', 'd223', 'd224', 'd258', 'd268', 'd277', 'd285', 'd299', 'd323'], 'daniel': ['d019', 'd026', 'd035', 'd048', 'd299'], 'quar': ['d299'], 'bracket': ['d224', 'd299'], 'clock': ['d068', 'd099', 'd186', 'd196', 'd219', 'd299'], 'display': ['d023', 'd028', 'd029', 'd052', 'd076', 'd078', 'd085', 'd107', 'd126', 'd128', 'd136', 'd150', 'd169', 'd170', 'd173', 'd194', 'd235', 'd254', 'd256', 'd281', 'd299', 'd301', 'd318', 'd323'], 'walker': ['d175', 'd182', 'd183', 'd254', 'd299'], 'art': ['d012', 'd014', 'd015', 'd016', 'd017', 'd018', 'd020', 'd026', 'd027', 'd029', 'd033', 'd042', 'd055', 'd058', 'd064', 'd066', 'd067', 'd077', 'd079', 'd085', 'd088', 'd101', 'd102', 'd103', 'd107', 'd109', 'd116', 'd119', 'd129', 'd136', 'd143', 'd156', 'd158', 'd159', 'd161', 'd170', 'd173', 'd174', 'd